In [1]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import os
import time
import warnings
import gc
gc.collect()
import os
from six.moves import urllib
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('seaborn')
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import StandardScaler

In [2]:
# to make this notebook's output stable across runs
np.random.seed(123)
gc.collect()
# To plot pretty figures
%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [3]:
#Reduce the memory usage - by Panchajanya Banerjee
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = reduce_mem_usage(pd.read_csv('train.csv',parse_dates=["first_active_month"]))
test = reduce_mem_usage(pd.read_csv('test.csv', parse_dates=["first_active_month"]))

In [ ]:
# Taking Reference from Other Kernels
def aggregate_transaction_new(trans, prefix):  
        
    agg_func = {
        'purchase_date' : ['max','min'],
        'month_diff' : ['mean', 'min', 'max'],
        'month_diff_lag' : ['mean', 'min', 'max'],
        'weekend' : ['sum', 'mean'],
        'authorized_flag': ['sum'],
        'category_1': ['sum','mean', 'max','min'],
        'purchase_amount': ['sum', 'mean', 'max', 'min'],
        'installments': ['sum', 'mean', 'max', 'min'],  
        'month_lag': ['max','min','mean'],
        'card_id' : ['size'],
        'month': ['nunique'],
        'hour': ['nunique'],
        'weekofyear': ['nunique'],
        'dayofweek': ['nunique'],
        'year': ['nunique'],
        'subsector_id': ['nunique'],
        'merchant_category_id' : ['nunique', lambda x:stats.mode(x)[0]],
        'merchant_id' : ['nunique', lambda x:stats.mode(x)[0]],
        'state_id' : ['nunique', lambda x:stats.mode(x)[0]],
        'city_id' : ['nunique', lambda x:stats.mode(x)[0]],
    }
    
    agg_trans = trans.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)
    
    df = (trans.groupby('card_id').size().reset_index(name='{}transactions_count'.format(prefix)))
    
    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')
    
    return agg_trans

In [8]:
new_transactions = reduce_mem_usage(pd.read_csv('new_merchant_transactions_clean_outlier.csv'))
new_transactions = new_transactions.loc[new_transactions.purchase_amount < 50,]
new_transactions['authorized_flag'] = new_transactions['authorized_flag'].map({'Y': 1, 'N': 0})
new_transactions['category_1'] = new_transactions['category_1'].map({'Y': 0, 'N': 1})

Mem. usage decreased to 106.71 Mb (52.5% reduction)


In [9]:
new_transactions['purchase_date'] = pd.to_datetime(new_transactions['purchase_date'])
new_transactions['year'] = new_transactions['purchase_date'].dt.year
new_transactions['weekofyear'] = new_transactions['purchase_date'].dt.weekofyear
new_transactions['month'] = new_transactions['purchase_date'].dt.month
new_transactions['dayofweek'] = new_transactions['purchase_date'].dt.dayofweek
new_transactions['weekend'] = (new_transactions.purchase_date.dt.weekday >=5).astype(int)
new_transactions['hour'] = new_transactions['purchase_date'].dt.hour 
new_transactions['quarter'] = new_transactions['purchase_date'].dt.quarter
new_transactions['is_month_start'] = new_transactions['purchase_date'].dt.is_month_start
new_transactions['month_diff'] = ((pd.to_datetime('01/03/2018') - new_transactions['purchase_date']).dt.days)//30
new_transactions['month_diff_lag'] = new_transactions['month_diff'] + new_transactions['month_lag']

In [31]:
new_transactions['Christmas'] = (pd.to_datetime('2017-12-26') - 
                                          new_transactions['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 16 else 0)
#Black Friday : 24th November 2017
new_transactions['BlackFriday'] = (pd.to_datetime('2017-11-27') - 
                                         new_transactions['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 8 else 0)

In [60]:
def aggregate_bymonth(trans, prefix):  
        
    agg_func = {
        'weekend' : ['sum', 'mean'],
        'purchase_amount': ['sum', 'mean', 'max', 'min'],
        'card_id' : ['size'],
        'subsector_id': ['nunique'],
        'merchant_category_id' : ['nunique', lambda x:stats.mode(x)[0]],
        'merchant_id' : ['nunique', lambda x:stats.mode(x)[0]],
        'state_id' : ['nunique', lambda x:stats.mode(x)[0]],
    }
    
    agg_trans = trans.groupby(['card_id','month','year']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)
    
    df = (trans.groupby('card_id').size().reset_index(name='{}transactions_count'.format(prefix)))
    
    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')
    
    return agg_trans

In [61]:
merge = aggregate_bymonth(new_transactions, prefix='new_')
merge = merge.drop(['new_transactions_count'], axis = 1)
merge.head()

,card_id,month,year,new_weekend_sum,new_weekend_mean,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_purchase_amount_min,new_card_id_size,new_subsector_id_nunique,new_merchant_category_id_nunique,new_merchant_category_id_<lambda>,new_merchant_id_nunique,new_merchant_id_<lambda>,new_state_id_nunique,new_state_id_<lambda>
0,C_ID_00007093c1,4,2018,0,0.000000,-1.328125,-0.664062,-0.656738,-0.671875,2,2,2,222,2,M_ID_00a6ca8a8a,2,2
1,C_ID_0001238066,3,2018,7,0.411765,-9.757812,-0.573730,-0.078308,-0.740723,17,7,13,278,16,M_ID_00a6ca8a8a,4,9
2,C_ID_0001238066,4,2018,5,0.555556,-5.093750,-0.565918,-0.371338,-0.718262,9,4,4,278,9,M_ID_4635824091,3,9
3,C_ID_0001506ef0,3,2018,0,0.000000,-1.447266,-0.723633,-0.715332,-0.731934,2,2,2,360,1,M_ID_ab756f937e,1,19
4,C_ID_0001793786,11,2017,6,0.285714,1.675781,0.079834,3.130859,-0.726562,21,11,16,278,21,M_ID_0360f86430,3,9


In [68]:
merge['Date'] = pd.to_datetime(merge[['year', 'month']].assign(Day=1))
a1 = merge.loc[merge.groupby('card_id').Date.idxmax(),:][['card_id','new_card_id_size','new_purchase_amount_sum','new_purchase_amount_mean']]

In [69]:
a2 = merge.loc[merge.groupby('card_id').Date.idxmin(),:][['card_id','new_card_id_size','new_purchase_amount_sum','new_purchase_amount_mean']]

In [70]:
a1.head()

,card_id,new_card_id_size,new_purchase_amount_sum,new_purchase_amount_mean
0,C_ID_00007093c1,2,-1.328125,-0.664062
2,C_ID_0001238066,9,-5.093750,-0.565918
3,C_ID_0001506ef0,2,-1.447266,-0.723633
5,C_ID_0001793786,10,-1.905273,-0.190552
7,C_ID_000183fdda,3,-1.479492,-0.493164


In [71]:
a2.head()

,card_id,new_card_id_size,new_purchase_amount_sum,new_purchase_amount_mean
0,C_ID_00007093c1,2,-1.328125,-0.664062
1,C_ID_0001238066,17,-9.757812,-0.573730
3,C_ID_0001506ef0,2,-1.447266,-0.723633
4,C_ID_0001793786,21,1.675781,0.079834
6,C_ID_000183fdda,8,-5.109375,-0.638672


In [ ]:
merge_new = aggregate_transaction_new(new_transactions, prefix='new_')
merge_new.head()

In [52]:
merge.columns

Index(['card_id', 'month', 'year', 'new_weekend_sum', 'new_weekend_mean',
       'new_purchase_amount_sum', 'new_purchase_amount_mean',
       'new_purchase_amount_max', 'new_purchase_amount_min',
       'new_installments_sum', 'new_installments_mean', 'new_installments_max',
       'new_installments_min', 'new_card_id_size', 'new_subsector_id_nunique',
       'new_merchant_category_id_nunique', 'new_merchant_category_id_<lambda>',
       'new_merchant_id_nunique', 'new_merchant_id_<lambda>',
       'new_state_id_nunique', 'new_state_id_<lambda>', 'new_city_id_nunique',
       'new_city_id_<lambda>', 'Date'],
      dtype='object')

In [6]:
new_transactions.columns

Index(['Unnamed: 0', 'authorized_flag', 'card_id', 'city_id', 'category_1',
       'installments', 'category_3', 'merchant_category_id', 'merchant_id',
       'month_lag', 'purchase_amount', 'purchase_date', 'category_2',
       'state_id', 'subsector_id', 'year', 'weekofyear', 'month', 'dayofweek',
       'weekend', 'hour', 'quarter', 'is_month_start', 'month_diff',
       'month_diff_lag'],
      dtype='object')

In [38]:
len(df)

25915

In [6]:
df = (new_transactions.groupby(['card_id','merchant_id','purchase_amount']).size().reset_index(name='count_hist'))
df['purchase_amount_hist'] = df.groupby(['card_id','merchant_id'])['purchase_amount'].transform('sum')
df['count_hist'] = df.groupby(['card_id','merchant_id'])['count_hist'].transform('sum')
df = df.drop_duplicates()
df = df.loc[df['count_hist'] >= 2]
agg_func = {
    'count_hist' : ['count'],
    'purchase_amount_hist':['sum','mean'],
}
df = df.groupby(['card_id']).agg(agg_func)
df.columns = [''.join(col).strip() for col in df.columns.values]
df.head(20)

,card_id,merchant_id,purchase_amount,count_hist
0,C_ID_00007093c1,M_ID_00a6ca8a8a,-0.656738,1
1,C_ID_00007093c1,M_ID_08f01305af,-0.671875,1
2,C_ID_0001238066,M_ID_00a6ca8a8a,-0.700195,1
3,C_ID_0001238066,M_ID_00a6ca8a8a,-0.676270,1
4,C_ID_0001238066,M_ID_0bb734e74a,-0.487061,1
5,C_ID_0001238066,M_ID_235e546dcc,-0.672363,1
6,C_ID_0001238066,M_ID_4635824091,-0.559570,1
7,C_ID_0001238066,M_ID_51b4a616bb,-0.470459,1
8,C_ID_0001238066,M_ID_6b8228d6a6,-0.656738,1
9,C_ID_0001238066,M_ID_7d8102bb34,-0.732910,1


In [14]:
new_transactions.loc[new_transactions.card_id == 'C_ID_0001238066']['purchase_amount'].sum()

-14.85

In [15]:
df = (new_transactions.groupby(['card_id','merchant_category_id','purchase_amount']).size().reset_index(name='count'))
df['purchase_amount'] = df.groupby(['card_id','merchant_category_id'])['purchase_amount'].transform('sum')
df['count'] = df.groupby(['card_id','merchant_category_id'])['count'].transform('sum')
df = df.drop_duplicates()
df['count_4'] = 0
df.loc[df['count'] >= 4, 'count_4'] = 1
df = df.loc[df['count'] >= 2]
df['mean4'] = 0
df.loc[df['count'] >= 4, 'mean4'] = df['purchase_amount']/df['count']
# df['purchase_amountx'] = 0
# df.loc[df['count'] >= 2, 'purchase_amountx'] = df['purchase_amount']/df['count']
agg_func = {
    'count' : ['count'],
    'count_4' : ['sum'],
    'purchase_amount':['sum','mean'],
    'mean4':['sum','mean']
}
df = df.groupby(['card_id']).agg(agg_func)
df.columns = [''.join(col).strip() for col in df.columns.values]
df.head(20)

,countcount,count_4sum,purchase_amountsum,purchase_amountmean,mean4sum,mean4mean
card_id,,,,,,
C_ID_0001238066,4,2,-8.262390,-2.065598,-1.179539,-0.294885
C_ID_0001793786,4,1,-2.167862,-0.541965,-0.465657,-0.116414
C_ID_000183fdda,1,0,-1.664307,-1.664307,0.000000,0.000000
C_ID_0002ba3c2e,1,0,-1.373535,-1.373535,0.000000,0.000000
C_ID_0002c7c2c1,1,0,-1.384277,-1.384277,0.000000,0.000000
C_ID_00032df08f,1,0,-1.164551,-1.164551,0.000000,0.000000
C_ID_0004c2a5ab,1,0,-1.179199,-1.179199,0.000000,0.000000
C_ID_000599daf9,1,0,-0.865601,-0.865601,0.000000,0.000000
C_ID_0007fe3157,1,0,-1.265137,-1.265137,0.000000,0.000000


In [ ]:
df = (new_transactions.groupby(['card_id','merchant_category_id']).size().reset_index(name='new_count'))
df['new_count'] = df.groupby(['card_id','merchant_category_id'])['new_count'].transform('sum')
df = df.drop_duplicates()
df = df.loc[df['new_count'] >= 2]
df = df.groupby(['card_id']).size().reset_index(name='new_count')
df.head(20)

In [ ]:
df = (new_transactions.groupby(['card_id','merchant_id']).size().reset_index(name='new_count'))
df['new_count'] = df.groupby(['card_id','merchant_id'])['new_count'].transform('sum')
df = df.drop_duplicates()
df = df.loc[df['new_count'] >= 2]
df = df.groupby(['card_id']).size().reset_index(name='new_count')
df.head(20)

In [ ]:
merge_new = aggregate_transaction_new(new_transactions, prefix='new_')

In [ ]:
merge_new.isnull().sum()

In [ ]:
merge_new.columns

In [ ]:
(merge_new.new_purchase_date_max - merge_new.new_purchase_date_min).dt.total_seconds()

In [ ]:
merge_new['freq'] = merge_new.new_transactions_count/(((merge_new.new_purchase_date_max - 
                                                         merge_new.new_purchase_date_min).dt.total_seconds())/86400)
merge_new['freq'] = pd.qcut(merge_new['freq'], 5)

In [ ]:
merge_new['freq']

In [14]:
new_transactions.purchase_date.min()

Timestamp('2017-03-01 03:24:51')

In [11]:
new_transactions.purchase_date.max()

Timestamp('2018-04-30 23:59:59')

In [27]:
df = new_transactions.groupby(['card_id','month','year','purchase_amount']).size().reset_index(name='month_count')
df['purchase_amount'] = df.groupby(['card_id','month','year'])['purchase_amount'].transform('sum')
df['month_count'] = df.groupby(['card_id','month','year'])['month_count'].transform('sum')
df = df.drop_duplicates()
df.head()

,card_id,month,year,purchase_amount,month_count
0,C_ID_00007093c1,4,2018,-1.328613,2
2,C_ID_0001238066,3,2018,-9.754944,17
19,C_ID_0001238066,4,2018,-5.095215,9
28,C_ID_0001506ef0,3,2018,-1.447266,2
30,C_ID_0001793786,11,2017,1.676027,21
